# Adaptive PDE discretizations on cartesian grids
## Volume : Divergence form PDEs
## Part : Applications
## Chapter : Heat and wave equations, in one dimension

We illustrate discretization of time dependent partial differential equations in dimension one: diffusion (in divergence form) and the wave equation, whose PDE formulation read 
$$
    \frac {\partial u}{\partial t} = \frac {\partial}{\partial x} \Big( c(x) \frac {\partial u}{\partial x}\Big),
    \quad \text{and} \quad
    \frac {\partial^2 u}{\partial t^2} = \frac {\partial}{\partial x} \Big( c(x) \frac {\partial u}{\partial x}\Big).
$$
For simplicity, we use periodic boundary conditions for the space variable, on the interval $\Omega = [0,1]$. The diffusion coefficient $c : \Omega \to ]0,\infty[$ is continuous and positive (it may also depend on time). 

We present the two most classical discretizations of the heat equations, explicit and implicit, their variational interpretation and stability analysis. We rely on the Hamiltonian formalism for the wave equation.

**Disclaimer.** This notebook does *not* contain original research. It is limited to elementary examples, and may serve as a gentle introduction to (some of) the numerical tools and techniques related to *time discretization*.
My original research is mainly devoted to the *spatial discretization* of PDE operators, which is irrelevant in here in dimension one. Examples in dimension two and higher, involving non-trivial geometrical constructions, can be easily produced by combining the contents of the other notebooks with this one, and may also be presented in subsequent notebooks.

**Related.** The discretization of time dependent *non-divergence* form PDEs is discussed [here](../Notebooks_NonDiv/Time1D_NonDiv.ipynb).

[**Summary**](Summary.ipynb) of volume Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Quadratic forms and their discretization](#1.-Quadratic-forms-and-their-discretization)
    * [1.1 Continuous setting](#1.1-Continuous-setting)
    * [1.2 Discrete setting](#1.2-Discrete-setting)
  * [1. The heat equation, explicit scheme](#1.-The-heat-equation,-explicit-scheme)
    * [1.1 Discretization using automatic differentiation](#1.1-Discretization-using-automatic-differentiation)
    * [1.2 Stability analysis](#1.2-Stability-analysis)
    * [1.3 Sparse matrix](#1.3-Sparse-matrix)
  * [2. Implicit scheme](#2.-Implicit-scheme)
    * [2.1 Discretization using automatic differentiation](#2.1-Discretization-using-automatic-differentiation)
  * [2.2 Stability analysis](#2.2-Stability-analysis)
    * [2.3 Sparse matrix](#2.3-Sparse-matrix)
  * [3. Wave equation](#3.-Wave-equation)
    * [3.1 Discretization using automatic differentiation](#3.1-Discretization-using-automatic-differentiation)
    * [2.2 Stability analysis](#2.2-Stability-analysis)
    * [3.3 Sparse matrices](#3.3-Sparse-matrices)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [6]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Time1D_Div','Div'))

In [7]:
from agd import FiniteDifferences as fd
from agd import AutomaticDifferentiation as ad
from agd.Plotting import animation_curve
norm_infinity = ad.Optimization.norm_infinity 
lo = ad.left_operand

In [8]:
import numpy as np
import scipy.sparse
import matplotlib.pyplot as plt
from matplotlib import rc; rc('animation', html='html5')

Some utility functions

In [9]:
def reload_packages():
    import importlib
    fd = importlib.reload(sys.modules['agd.FiniteDifferences'])
    ad = importlib.reload(sys.modules['agd.AutomaticDifferentiation'])
    ad.reload_submodules()

In [10]:
#from itertools import accumulate # Accumulate with initial value only exists in Python >= 3.8
def accumulate(iterable, func, initial):
    yield initial
    for element in iterable:
        initial = func(initial, element)
        yield initial

## 1. Quadratic forms and their discretization

We introduce the basic objects underlying the heat and wave equations, which are quadratic forms on suitable spaces of functions. We then introduce their discretization.

### 1.1 Continuous setting

We briefly recall some of the concepts underlying divergence form PDE discretizations. The objective here is to set notations, not present a course on the subject, and not to achieve absolute mathematical rigor either. Please consider a textbook for that purpose.

**Quadratic forms, polarization and associated operator.**
A bilinear symmetric form is a function $Q(u,v)$ of all elements $u,v$ of a vector space $V$, which is linear w.r.t 
$u$ and $v$ (separately, not simultaneously), and obeys the symmetry $Q(u,v)=Q(v,u)$ for all $u,v \in V$.

A quadratic form is the specialization $Q(u,u)$ of a bilinear symmetric form to the case $u=v$, $u\in V$. No information is lost in this process, since the bilinear symmetric form can be recovered using an identity referred to as polarization. The quadratic form is said non-negative if $Q(u,u)\geq 0$ for all $u\in V$.

One can associate a linear operator to a quadratic form $Q$ whose domain $V$ is a (dense subset of a) Hilbert space $H$. This operator is denoted by the same letter, and defined by the identity
$$
    <u,Qv> = Q(u,v)
$$
for all $u,v \in V$. 

**Case of the $H^1$ semi-norm.** We denote by $Q$ the Dirichlet elliptic energy, defined for all $u \in H^1(\Omega)$ by 
$$
    Q(u,u) := \int_\Omega c(x)|\nabla u(x)|^2 \ dx.
$$
Note that $Q$ is non-negative, since $c\geq 0$.
The ambient space is $L^2(\Omega)$, equipped with the scalar product defined by 
$$
    <u,v> = \int_\Omega u(x)v(x) \ dx.
$$

**PDE reformulation.**
An integration by parts, recalling that $\Omega=[0,1]$ with periodic b.c., shows that 
$$
    Q u = - \frac {\partial}{\partial x} \Big( c(x) \frac {\partial u}{\partial x}\Big)
$$
Therefore the heat and wave equations can be reformulated as 
$$
    \frac d {dt} u = - Q u,
    \quad \text{and} \quad
    \frac {d^2} {dt^2} u = - Q u. 
$$

### 1.2 Discrete setting

For any $h>0$, which is the inverse of a positive integer, we introduce the set $\Omega_h := \Omega\cap hZ$. In the numerical codes, $h$ is denoted `dx`.

**Discretization.**
Define the quadratic form $Q_h$ by 
$$
    Q_h(u,u) := h \sum_{x \in \Omega_h} \frac{c(x)} 2\Big[\big(\frac {u(x+h)-u(x)} h\big)^2+\big(\frac {u(x-h)-u(x)} h\big)^2\Big]
$$
as well as $I_h$ defined by 
$$
    I_h(u,u) := h \sum_{x \in \Omega_h} u(x)^2.
$$

**Consistency.** For twice continuously differentiable $u,v:\Omega \to R$, one has second order consistency
$$
    Q_h(u,v) = Q(u,v) + O(h^2)
    \quad \text{and} \quad
    I_h(u,v) = <u,v> + O(h^2)
$$

**Ambient scalar product.**
The vector space $V_h$ of all real functions on $\Omega_h$ is a Hilbert space, which could be equipped with the scalar product $I_h$. It could also be identified with $R^N$, where $N = \#(\Omega_h)$ is the cardinality of the discrete domain, and equipped with the usual Euclidean scalar product. In our setting, these two scalar products differ by the multiplicative factor $h>0$.

The first convention is the closest to the continuous formulation of the problem, whereas the second convention is more natural in the context of a numerical implementation. We opt for the second convention, which causes the introduction of the operator $I_h$ in the discretization, but is also simpler from some points of view.

Let us introduce the discretizations of the discretized operators $Q_h$ and $I_h$.

In [11]:
def Q(u,v,c,dx):
    """
    Finite differences discretization of the H1 bilinear form,
    in dimension 1, with periodic b.c.
    """
    dup = fd.DiffUpwind(u,( 1,),dx,padding=None)
    dvp = fd.DiffUpwind(v,( 1,),dx,padding=None)

    dum = fd.DiffUpwind(u,(-1,),dx,padding=None)
    dvm = fd.DiffUpwind(v,(-1,),dx,padding=None)

    r = 0.5*c*(dup*dvp+dum*dvm)
    return r.sum(axis=0)*dx

def I(u,v,dx):
    """Approximation of the L2 scalar product"""
    r=u*v
    return r.sum(axis=0)*dx

For concreteness, we introduce a discretization grid, some diffusion coefficients, and test functions.

In [12]:
X,dx = np.linspace(0,1,100,endpoint=False,retstep=True)
Tmax = 0.5 # Default time interval is [0,Tmax]

In [13]:
pi2 = np.pi*2.
c_constant = 0.7
c_positive = 0.7  + 0.4*np.sin(pi2*X)

u_disc = 1.*(X>=0.5)*(X<=0.75)
u_cont = np.maximum(0.,(0.5-X)*(X-0.75)); u_cont/=np.max(u_cont)
u_smooth = u_cont**2

We can test our bilinear forms on arbirary numpy arrays.

In [14]:
u_ = np.sin(pi2*X)
v_ = np.cos(pi2*X+1)
Q(u_,v_,c_positive,dx), I(u_,v_,dx)

(-11.623155409290282, -0.42073549240394836)

Specializing to $u=v$ yields a non-negative value, by construction.

In [15]:
Q(u_,u_,c_positive,dx), I(u_,u_,dx)

(13.812901002099066, 0.5)

## 1. The heat equation, explicit scheme


**Continuous setting.** The heat equation can be written in the so-called variational form
$$
    <\frac d {dt} u, v> = - Q(u,v)
$$
for all test functions $v$. Recall that $<,>$ denotes the $L^2$ scalar product, and $Q$ the bilinear form associated with the Dirichlet energy.

**Discretization.**
We use a first order explicit time discretization, which yields the system
$$
    I_h\big(\frac{u_{n+1}-u_n}{\delta t},v\big) = - Q_h(u_n,v)
$$
for all test functions $v$, where $u,v,w : \Omega_h \to R$.

### 1.1 Discretization using automatic differentiation

As a first approach, we rely on automatic differentiation to implement the variational formulation directly.

In [16]:
def HeatExplicit(u,c,dx,dt):
    """One time step of the explicit scheme for the heat equation"""
    
    # Define independent second order AD variables
    u_ad = ad.Sparse2.identity(u.shape) # Unknown
    v_ad = ad.Sparse2.identity(u.shape,shift=u.size) # Test function
    
    dtu_ad = (u_ad-u)/dt # Time derivative
    
    weakform = I(dtu_ad,v_ad,dx) + Q(u,v_ad,c,dx) # Should vanish for all test functions v_ad
    
    return weakform.solve_weakform() # Finds u_ad

In [17]:
dt = 4e-5
solution =np.array(list(accumulate(
    np.arange(0,100*dt,dt), # time interval
    initial=u_disc, # initial condition
    func=lambda u,t: HeatExplicit(u,c_positive,dx,dt) # evolution rule
)))

In [18]:
animation_curve(X,solution)

### 1.2 Stability analysis

We can rephrase the update of the explicit scheme for the heat equation in the form
$$
    u_{n+1} = ({\rm Id} - I_h^{-1} Q_h \delta t) u_n.
$$
where ${\rm Id}$ denotes the identity matrix.

The scheme is stable, in the $L^2$ norm, iff all the eigenvalues of ${\rm Id} - I_h^{-1} Q_h \delta t$ lie in $[-1,1]$. Equivalently iff, in the sense of symmetric matrices
$$
Q_h \delta t \preceq 2 I_h.
$$
Said otherwise, one must have $Q_h(u,u) \delta t \preceq 2 I_h(u,u)$ for any $u : \Omega_h \to R$. Based on the inequality $(a-b)^2 \leq 2(a^2+b^2)$, and some simplifications, we obtain
$$
    Q_h(u,u) 
    \leq h^{-1} \sum_{x \in \Omega_h} c(x) \big[u(x+h)^2+2 u(x)^2 +u(x-h)^2\big] 
    \leq \frac {4c_{\max}} h \sum_{x \in \Omega_h} u(x)^2 = \frac {4c_{\max}} {h^2} I_h(u,u),
$$
where $c_{\max}$ is a uniform bound for the diffusion coefficient $c$. This results in the CFL condition
$$
    2c_{\max}\delta t \leq h^2.
$$

<!---
Denote $w=(u_{n+1}-u_n)/\delta t$, where $u_n,u_{n+1} : \Omega_h\to R$ are successive time steps of the scheme, and recall from the variational formulation that
$$
    <w,I_h v> = - <v,Q_h u_n>
$$
for all test functions $v : \Omega_h \to R$.
Choosing applying twice this identity we obtain
$$
    < w,I_h w > = - <w, Q_h u_n> = <I_h^{-1} Q_h u_n, Q_h u_n>
$$
where we successively choose $v=w$ and $v=I_h^{-1} Q_h u_n$.

Denote $C_h$ such that $\|Q_h v\| \leq C_h v$ for all $v$, and recall that $I_h v = h v$. We obtain from the previous equation
$$
    \|u_{n+1} - u_n\| \leq (C_h \delta t/h) \|u_n\|,
$$
which yields stability provided $C_h \delta t/h \leq K \delta t$ for some $K \geq 0$.
--->

In [19]:
def HeatExplicit_CFL(c,dx):
    """
    The explicit discretization of the heat equation
    is stable provided the time step dt is below this threshold
    """
    return dx**2/(2*np.max(c))

In [20]:
HeatExplicit_CFL(c_positive,dx)

4.545454545454545e-05

### 1.3 Sparse matrix 

The implementation presented above of the heat equation lacks in numerical efficiency, and perhaps explicitness.
To remedy this problem, we extract the matrices involved, and replace the weak form solution with an explicit matrix-vector product.

Let us extract the (sparse symmetric) matrix associated with quadratic form $Q_h$.
Note the $1/2$ factor, as the matrix of interest is *half the hessian* of $Q_h(u,u)$.

In [21]:
u_ad = ad.Sparse2.identity(X.shape)
Quu_ad = 0.5*Q(u_ad,u_ad,c_positive,dx) #Note 1/2 factor
Q_mat = scipy.sparse.coo_matrix(Quu_ad.triplets()).tocsc() # Hessian matrix

The numerical solution is computed much faster, but is equal up to floating point rounding error.

In [22]:
dt = 4e-5
solution2 = np.array(list(accumulate(
    np.arange(0,100*dt,dt), # time interval
    initial=u_disc, # initial condition
    func=lambda u,t: u-(dt/dx)*Q_mat*u # evolution rule
)))

In [23]:
assert norm_infinity(solution-solution2) < 1e-14

## 2. Implicit scheme

**Continuous setting.**
The heat equation can be regarded as the gradient flow of the energy $\frac 1 2 Q(u,u)$ w.r.t. the $L^2$ metric. 
A semi-discretized formulation reflecting this principle is as follows: define $u_{n+1}$ as the minimizer to 
$$
     \min_{u_{n+1}} \frac 1 {\delta t} \|u_{n+1}-u_n\|^2 + Q(u_{n+1},u_{n+1}).
$$
There exists another, non-linear, similarly looking formulation of the heat equation: as the gradient flow of the entropy w.r.t the Wasserstein metric.

Each time step of the implicit scheme involves solving a linear equation, which is more expensive than the matrix-vector products involved in the explicit scheme. However, the time steps used in the implicit scheme are unconstrained, and the scheme is unconditionally stable.

### 2.1 Discretization using automatic differentiation

We translate in the discrete setting the variational principle defining $u_{n+1}$, in the form
$$
    \min_{u_{n+1}} \frac 1 {\delta t} I_h(u_{n+1}-u_n,u_{n+1}-u_n) + Q_h(u_{n+1},u_{n+1}).
$$

In [24]:
def HeatImplicit(u,c,dx,dt):
    u_ad = ad.Sparse2.identity(u.shape)
    energy = (1/dt)*I(u_ad-u,u_ad-u,dx) + Q(u_ad,u_ad,c,dx)
    return energy.solve_stationnary()

As announced, one of the key interests of the implicit scheme is the ability to take large time steps, here several orders of magnitude larger than for the explicit scheme.

In [25]:
dt = 1e-2
solution =np.array(list(accumulate(
    np.arange(0,20*dt,dt), # time interval
    initial=u_disc, # initial condition
    func=lambda u,t: HeatImplicit(u,c_positive,dx,dt) # evolution rule
)))

In [26]:
animation_curve(X,solution)

## 2.2 Stability analysis

We can rephrase the update of the explicit scheme for the heat equation in the form
$$
    u_{n+1} = ({\rm Id} + I_h^{-1} Q_h \delta t)^{-1} u_n.
$$
where ${\rm Id}$ denotes the identity matrix. 

The scheme is stable provided all the eigenvalues of the operator $A = ({\rm Id} + I_h^{-1} Q_h \delta t)^{-1}$ lie in $[-1,1]$. This property holds unconditionally w.r.t $\delta t \geq 0$. Indeed, the eigenvalues of $A$ take the form
$$
    \frac 1 {1+\lambda h/\delta t}
$$
where $\lambda$ is an eigenvalue of $Q_h$, hence $\lambda \geq 0$ since $Q_h$ is a non-negative symmetric matrix. (We use the fact that $I_h = h {\rm Id}$.)

### 2.3 Sparse matrix

The implicit scheme for the heat equation involves solving repeatedly the same (sparse, positive definite) linear system. This procedure can be made numerically more efficient by pre-factorization of the matrix.

In [27]:
impl_mat = scipy.sparse.eye(len(u_disc)) + (dt/dx)*Q_mat
solver = scipy.sparse.linalg.factorized(impl_mat.tocsc())

The scheme runs faster, but yields an identical solution up to machine precision.

In [28]:
dt = 1e-2
solution2 = np.array(list(accumulate(
    np.arange(0,20*dt,dt), # time interval
    initial=u_disc, # initial condition
    func=lambda u,t: solver(u) # evolution rule
)))

In [29]:
assert norm_infinity(solution-solution2) < 1e-13

## 3. Wave equation

**Continuous setting.**
The wave equation can be rephrased in the Hamiltonian formalism
$$
    \frac {d u} {dt} = \frac{\partial H}{\partial \dot u} 
    \quad \text{and} \quad 
    \frac {d\dot u} {dt} = - \frac{\partial H}{\partial u}.
$$
The Hamiltonian is defined as the sum of a potential *elastic* energy, and of a kinetic energy
$$
    H(u,\dot u) := \frac 1 2 \big( Q(u,u) + <\dot u,\dot u> \big).
$$
The Hamitlonian is constant in time, along solutions of sufficient regularity.
The discretization mimicks this formulation, using a symplectic integrator to conserve the Hamiltonian and ensure stability.

### 3.1 Discretization using automatic differentiation

**Translation of the continuous setting.**
We introduce a discretized hamiltonian, defined by
$$
    H^0_h(u,\dot u) := \frac 1 2 \big(Q_h(u,u) + I_h(\dot u,\dot u) \big).
$$
The Euler symplectic integrator is defined as 
$$
    \frac{u_{n+1}-u_n}{\delta t} = I_h^{-1}\frac{\partial H^0_h}{\partial \dot u}(u_n,\dot u_n)
    \quad \text{and} \quad
    \frac{\dot u_{n+1}-\dot u_n}{\delta t} = - I_h^{-1}\frac{\partial H^0_h}{\partial u}(u_{n+1},\dot u_{n+1}),
$$
where $u_n,v_n,u_{n+1},v_{n+1} : \Omega_h \to R$.
We regard $u_n,v_n,u_{n+1},v_{n+1}$ are regarded as vectors in $R^N$, $N = \#(\Omega_h)$, and differentiate $H^0_h$ as a common function on $R^{2N}$. The $I_h^{-1}$ factor arises from this differentiation, in the standard euclidean sense, rather than w.r.t the $I_h(,)$ scalar product.

**Rescaling.**
For convenience, we recall that $I_h = h {\rm Id}$, and incorporate this factor in the discretized Hamiltonian. Denoting
$$
    H_h(u,\dot u) := h^{-1} H^0_h(u,\dot u)
$$
one now has the standard form of Euler's symplectic scheme
$$
    \frac{u_{n+1}-u_n}{\delta t} = \frac{\partial H_h}{\partial \dot u}(u_n,\dot u_n)
    \quad \text{and} \quad
    \frac{\dot u_{n+1}-\dot u_n}{\delta t} = -\frac{\partial H_h}{\partial u}(u_{n+1},\dot u_{n+1}),
$$
For a general Hamiltonian, the second step would be implicit, see for instance this [notebook](../../Notebooks_Algo/Dense.ipynb). However, the wave equation hamiltonian benefits from a separable structure, which makes the two updates explicit.

In [30]:
def WaveHamiltonian(u,up,c,dx):
    return ( lo(Q(u,u,c,dx)) + I(up,up,dx) )/(2*dx) 

We apply automatic differentiation to the Hamiltonian, and implement Euler's symplectic scheme.
We introduced the `lo = ad.left_operand` function in the Hamiltonian for AD compatibility, see notebook [ADBugs](../../Notebooks_Algo/ADBugs.ipynb).

In [31]:
def SeparableSymplectic(q,p,H,dt):
    """Euler's symplectic integrator, for a Hamiltonian assumed to be separable"""
    p_ad = ad.Sparse.identity(constant=p)
    q_next = q+dt*H(q,p_ad).to_dense().gradient()
    
    # Using the separable structure here (p instead of p_next in r.h.s)
    q_ad = ad.Sparse.identity(constant=q_next)
    p_next = p-dt*H(q_ad,p).to_dense().gradient() 
    return q_next,p_next

Since the wave equation does not have a regularizing effect, unlike the heat equation, we use a continuous initial condition.

In [32]:
def H(q,p): return WaveHamiltonian(q,p,c_positive,dx)
dt = 9e-3
solution = np.array(list(accumulate(
    np.arange(0,200*dt,dt), # time interval
    initial=(u_cont,0.*u_cont), # initial condition
    func=lambda state,t: SeparableSymplectic(*state,H,dt) # state contains q and p
)))

In [33]:
animation_curve(X,solution[:,0,:])

### 2.2 Stability analysis

**Separable quadratic Hamiltonians.**
Symplectic integrators preserve a perturbation of the Hamiltonian, and this property is the key to stability analysis. We briefly present this theory in the case of separable quadratic Hamilonians on $R^N \times R^N$, which is enough for our purposes. Assume that 
$$
    H(q,p) = \frac 1 2 ( <q,Aq> + <p,Bp>),
$$
where $A$ and $B$ are symmetric matrices. Euler's symplectic scheme yields
$$
    q_{n+1} = q_n + \delta t B p_n,
    \quad \text{and} \quad
    p_{n+1} = p_n - \delta t A q_n.
$$
An elementary computation shows that the following quantity is *exactly* conserved along the iterations
$$
    \tilde H(q,p) := \frac 1 2 ( <q,Aq> + <p,Bp> + \delta t <Aq,Bp>)
$$
If $\tilde H$ is a positive definite quadratic form, then the iterates $(q_n,p_n)_{n\geq 0}$ are bounded independently of $n$. This condition amounts to 
$$
    \begin{pmatrix}
    A & \frac {\delta t} 2 A B\\
    \frac {\delta t} 2 B A & B
    \end{pmatrix}
    \succ 0
$$
where $M\succ N$ means that $M-N$ is positive definite. If $A$ and $B$ commute, then this condition can be simplified to 
$$
    A \succ 0, \quad 
    B \succ 0, \quad 
    \delta t^2 AB \prec 4 {\rm Id}.
$$

**Specialization to the wave equation**
We neglect here the technicalities associated with positive definiteness versus semi-definiteness.
The operators $Q_h$ and $I_h$ are positive (semi-)definite and commute, thus only the third condition remains. After specialization we obtain the stability criterion
$$
    \delta t^2 Q_h \prec 4 I_h.
$$
(We used the identity $I_h = h {\rm Id}$.) From the same analysis as in the explicit Euler equation, based on  the specific form of $Q_h$, we obtain the condition
$$
    \delta t^2 c_{\max} \leq \delta x^2.
$$
(Note that the wave velocity is $\sqrt{c_{\max}}$.)

In [34]:
def Wave_CFL(c,dx):
    return dx/np.sqrt(np.max(c))

In [35]:
Wave_CFL(c_positive,dx),dt

(0.009534625892455923, 0.009)

In [36]:
def ConservedHamiltonian(u,up,c,dx,dt):
    """
    Perturbation of the discretized Wave Hamiltonian, 
    which is exactly conserved by Euler's symplectic scheme.
    """
    return WaveHamiltonian(u,up,c,dx) + dt*Q(u,up,c,dx)/(2*dx)

In [37]:
invariant = [ConservedHamiltonian(u,up,c_positive,dx,dt) for u,up in solution]
assert norm_infinity(np.array(invariant)-invariant[0])<1e-12

### 3.3 Sparse matrices

We improve the computational efficiency, and explicitness, of the numerical scheme by using sparse matrix-vector products instead of relying on automatic differentiation.

In [38]:
dt = 9e-3
def WaveSymplectic(u,up): 
    u_next = u+dt*up
    up_next = up-(dt/dx)*Q_mat*u_next # Q_mat is the matrix of Q_h with c_positive
    return u_next,up_next 

solution2 = np.array(list(accumulate(
    np.arange(0,200*dt,dt), # time interval
    initial=(u_cont,0.*u_cont), # initial condition
    func=lambda state,t: WaveSymplectic(*state) # evolution rule
)))

In [39]:
assert norm_infinity(solution-solution2)<1e-12